In [1]:
import pandas as pd
from scipy import spatial
import numpy as np
import geopy.distance
import timeit

In [2]:
#Read in port locations and distances
df_distances = pd.read_csv('Data/path/distances.csv')
df_ports = pd.read_csv('Data/path/ports.csv')
df_routes = pd.read_csv('Data/path/routes.csv')

#Read in level 1 solar plant locations [lon, lat, kWh/kWp yearly, average W/m2 yearly]
df_ren = pd.read_csv('Data/level1renewable.csv')

In [3]:
df_ports.sample(10)

,Unnamed: 0,PORT_NAME,INDEX_NO,coords
2089,6914,NaN,NaN,"((39.14857738764045, 21.180353333333336),)"
1461,3842,PROVIDENCE,7420.0,"((-71.37825875305627, 41.794920000000005),)"
64,536,Chemainus-Ladysmith,NaN,"((-123.72890913870589, 48.995791962),)"
1791,5441,RADA PUNTA ARENAS,14100.0,"((-70.80099704966483, -52.95232),)"
811,2690,MATSUYAMA,62030.0,"((132.67711422461986, 33.807995),)"
1605,4094,lulea,NaN,"((22.30952747604241, 65.540555),)"
1118,3217,PELABUHAN SUNGAI UDANG,49962.0,"((102.11265, 2.2323500000000003),)"
1729,4764,vavouto,NaN,"((164.6732166317222, -21.003845),)"
22,226,SHINGU,61967.0,"((135.98211007433795, 33.67201162967831),)"
2058,6875,NaN,NaN,"((43.109601412532314, 11.678745833333334),)"


In [4]:
lons = df_ren['Longitude']
lats = df_ren['Latitude']

In [5]:
coords = df_ports['coords'].values.tolist()
coords = [i.strip('()') for i in coords]
coords = [i.strip("'),'") for i in coords]
coords = [i.split(', ') for i in coords]

coords2=[]
for i in range(len(coords)):
    li=[]
    for j in range(2):
                li.append(float(coords[i][j]))
    coords2.append(li)    

In [6]:
pt = [6, 30]  # <-- the point to find

nearest = coords2[spatial.KDTree(coords2).query(pt)[1]] # <-- the nearest point 

distance,index = spatial.KDTree(coords2).query(pt)

print(distance)
print(index)
coords2[index]


5.671611487518396
1815


[10.107777625000002, 33.910669999999996]

In [13]:
gen_coords = zip(lons,lats)
#closest = []
longs = []
latis = []
distance = []
port_code = []

for i in gen_coords:
    nearest = coords2[spatial.KDTree(coords2).query(i)[1]] # <-- the nearest point 
    distanceKD,index = spatial.KDTree(coords2).query(i)
    
    #closest.append(coords2[index])
    longs.append(coords2[index][0])
    latis.append(coords2[index][1])
    port_code.append(df_ports.loc[index,'Unnamed: 0'])
    distance.append(geopy.distance.distance((reversed(i)),(reversed(coords2[index]))))
    
df_ren['Port Code'] = port_code
df_ren['Port Long.'] = longs
df_ren['Port Lat.'] = latis
df_ren['Gen-Port Direct Dist.'] = distance

In [14]:
driving_distance = []

In [15]:
#Don't re-run unless necessary - getting driving distance from all gen. points to closest port. Takes about 50 mins

for i in range(0,len(df_ren['Longitude'])):
    try:
        # call the OSMR API
        r = requests.get(f"http://router.project-osrm.org/route/v1/car/{df_ren.loc[i,'Longitude']},{df_ren.loc[i,'Latitude']};{df_ren.loc[i,'Port Long.']},{df_ren.loc[i,'Port Lat.']}?overview=false""")
        # then you load the response using the json libray
        # by default you get only one alternative so you access 0-th element of the `routes`
        routes = json.loads(r.content)
        route_1 = routes.get("routes")[0]
        driving_distance.append(route_1["distance"])
    except:
        driving_distance.append(None)

df_ren['Gen-Port Driving Dist.'] = driving_distance

In [16]:
df_ren.to_csv('Data/renewables.csv')

In [17]:
df_ren = pd.read_csv('Data/renewables.csv')

In [12]:
df_ren

,Unnamed: 0,Longitude,Latitude,Solar Energy Potential,Wind Power Density,Port Code,Port Long.,Port Lat.,Gen-Port Direct Dist.,Gen-Port Driving Dist.
0,0,-159.0,56.0,1063.974986,472.33026,4944,57.783555,-152.415736,447.5045175597334 km,NaN
1,1,-159.0,59.0,1079.305006,369.35553,4944,57.783555,-152.415736,408.02024289926123 km,NaN
2,2,-135.0,57.0,917.975038,958.34094,3146,55.902870,-130.000127,331.53387693717167 km,NaN
3,3,-128.0,55.0,999.004996,1433.11040,3679,53.977895,-128.671822,121.8188056016047 km,NaN
4,4,-126.0,55.0,1163.984960,77.40454,3679,53.977895,-128.671822,207.16071533791597 km,NaN
...,...,...,...,...,...,...,...,...,...,...
5965,5965,174.0,-36.0,1325.315015,343.84467,2198,-35.865232,174.539025,50.88832144926866 km,NaN
5966,5966,175.0,-39.0,1227.860030,146.18901,2488,-38.174366,174.652134,96.5329347741636 km,NaN
5967,5967,175.0,-38.0,1220.925010,331.73700,2488,-38.174366,174.652134,36.13743745041353 km,NaN
5968,5968,175.0,-37.0,1302.685002,310.05020,6825,-36.840815,174.767247,27.243295374351717 km,NaN
